<a href="https://colab.research.google.com/github/oserikov/data-science-nlp/blob/master/1_%D0%BC%D0%BE%D1%80%D1%84%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -qq install yargy --progress-bar off
!pip -qq install pymorphy2 --progress-bar off
!pip -qq install -U PyYAML --progress-bar off
!pip -qq install rnnmorph --progress-bar off
!pip -qq install rusenttokenize --progress-bar off
!pip -qq install pymystem3==0.1.10

import nltk
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)




import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression

from collections import defaultdict as dd
from operator import itemgetter
from pymorphy2.tokenizers import simple_word_tokenize as word_tokenize
from nltk.corpus import stopwords
from rusenttokenize import ru_sent_tokenize
import string
import pymorphy2
from rnnmorph.predictor import RNNMorphPredictor


import warnings
def warn(*args, **kwargs):
    pass

warnings.warn = warn

Using TensorFlow backend.


## Морфологический анализ

Задачи морфологического анализа:

* Разбор слова — определение нормальной формы (леммы), основы (стема) и грамматических характеристик слова
* Синтез словоформы — генерация словоформы по заданным грамматическим характеристикам из леммы

Морфологический анализ — не самая сильная сторона NLTK.

## POS-tagging

**Частеречная разметка**, или **POS-tagging** _(part of speech tagging)_ —  определение части речи и грамматических характеристик слов в тексте (корпусе) с приписыванием им соответствующих тегов.

Для большинства слов возможно несколько разборов (т.е. несколько разных лемм, несколько разных частей речи и т.п.). Теггер генерирует  все варианты, ранжирует их по вероятности и по умолчанию выдает наиболее вероятный. Выбор одного разбора из нескольких называется **снятием омонимии**, или **дизамбигуацией**.

### Наборы тегов

Существует множество наборов грамматических тегов, или тегсетов:
* НКРЯ
* Mystem
* UPenn
* OpenCorpora (его использует pymorphy2)
* Universal Dependencies
* ...

Есть даже [библиотека](https://github.com/kmike/russian-tagsets) для преобразования тегов из одной системы в другую для русского языка, `russian-tagsets`. Но важно помнить, что преобразования бывают с потерями.

На данный момент стандартом является **Universal Dependencies**. Подробнее про проект можно почитать [вот тут](http://universaldependencies.org/), а про теги — [вот тут](http://universaldependencies.org/u/pos/). Вот список основных (частереных) тегов UD:

* ADJ: adjective
* ADP: adposition
* ADV: adverb
* AUX: auxiliary
* CCONJ: coordinating conjunction
* DET: determiner
* INTJ: interjection
* NOUN: noun
* NUM: numeral
* PART: particle
* PRON: pronoun
* PROPN: proper noun
* PUNCT: punctuation
* SCONJ: subordinating conjunction
* SYM: symbol
* VERB: verb
* X: other



### pymystem3

**pymystem3** — это питоновская обертка для яндексовского морфологичекого анализатора Mystem. Его можно скачать отдельно и использовать из консоли. Может работать с незнакомыми словами (out-of-vocabulary words, OOV).


* [Документация Mystem](https://tech.yandex.ru/mystem/doc/index-docpage/)
* [Документация pymystem3](http://pythonhosted.org/pymystem3/)

Инициализируем Mystem c дефолтными параметрами. А вообще параметры есть такие:
* mystem_bin - путь к `mystem`, если их несколько
* grammar_info - нужна ли грамматическая информация или только леммы (по дефолту нужна)
* disambiguation - нужно ли снятие омонимии - дизамбигуация (по дефолту нужна)
* entire_input - нужно ли сохранять в выводе все (пробелы всякие, например), или можно выкинуть (по дефолту оставляется все)

Методы Mystem принимают строку, токенизатор вшит внутри. Можно, конечно, и пословно анализировать, но тогда он не сможет учитывать контекст.

In [2]:
from pymystem3 import Mystem

# сохраняем класс в переменную
mystem = Mystem() 

text = """Система состоит из камеры и программного обеспечения, которое анализирует фотографию.
 Суть технологии — сопоставление лиц, попавших в объектив, с изображениями из базы данных"""

print(text)
print(mystem.lemmatize(text))

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


Система состоит из камеры и программного обеспечения, которое анализирует фотографию.
 Суть технологии — сопоставление лиц, попавших в объектив, с изображениями из базы данных
['система', ' ', 'состоять', ' ', 'из', ' ', 'камера', ' ', 'и', ' ', 'программный', ' ', 'обеспечение', ', ', 'который', ' ', 'анализировать', ' ', 'фотография', '.', '\n', ' ', 'суть', ' ', 'технология', ' — ', 'сопоставление', ' ', 'лицо', ', ', 'попадать', ' ', 'в', ' ', 'объектив', ', ', 'с', ' ', 'изображение', ' ', 'из', ' ', 'база', ' ', 'данные', '\n']


In [3]:

# метод .analyze() даст грамматическую информацию о словах
words_analized = mystem.analyze(text)

print('Слово - ', words_analized[0]['text'])
print('Разбор слова - ', words_analized[0]['analysis'][0])
print('Лемма слова - ', words_analized[0]['analysis'][0]['lex'])
print('Грамматическая информация слова - ', words_analized[0]['analysis'][0]['gr'])

Слово -  Система
Разбор слова -  {'lex': 'система', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}
Лемма слова -  система
Грамматическая информация слова -  S,жен,неод=им,ед


###  pymorphy2

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). Может работать с незнакомыми словами (out-of-vocabulary words, OOV).

[Документация pymorphy2](https://pymorphy2.readthedocs.io/en/latest/)

### rnnmorph
**rnnmorph** &mdash; это морфологический анализатор на нейросетях, занявший первое место на дорожке по морофологическому анализу ["Диалога 2017"](http://www.dialog-21.ru/evaluation/2017/morphology/). 

Предлагает меньшее количество тегов в разборе.

Работает заметно медленнее, чем pymorphy и mystem.

[Документация rnnmorph](https://github.com/IlyaGusev/rnnmorph)

### maru
**maru** — это морфологический анализатор на нейросетях, в документации указаны результаты чуть лучше, чем у rnnmorph на той же задаче.

Тоже медленный

[Документация maru](https://github.com/chomechome/maru)

### Классификация новостей Ленты по темам

In [0]:
! wget -q -O lenta-ru-news-part.csv https://www.dropbox.com/s/ja23c9l1ppo9ix7/lenta-ru-news-part.csv?dl=0

In [5]:
lenta = pd.read_csv('lenta-ru-news-part.csv', usecols=['title', 'text', 'topic'])
lenta["topic_cat"] = lenta.topic.astype('category').cat.codes

def reload_lenta_dataset():
    global lenta
    lenta = pd.read_csv('lenta-ru-news-part.csv', usecols=['title', 'text', 'topic'])
    lenta = lenta.sample(frac=1).reset_index(drop=True).dropna(subset = ['text', 'topic'])
    lenta["topic_cat"] = lenta.topic.astype('category').cat.codes



lenta.head()

,title,text,topic,topic_cat
0,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,3
1,Овечкин повторил свой рекорд,Капитан «Вашингтона» Александр Овечкин сделал...,Спорт,3
2,Названы регионы России с самым дорогим и дешев...,Производитель онлайн-касс «Эвотор» проанализир...,Экономика,4
3,Россию и Украину пригласили на переговоры по газу,Вице-президент Еврокомиссии Марош Шефчович при...,Экономика,4
4,Хоккеист НХЛ забросил шайбу с отрицательного угла,Нападающий клуба «Эдмонтон Ойлерс» Коннор Макд...,Спорт,3


In [6]:
lenta.topic.value_counts()

Экономика          79538
Спорт              64421
Культура           53803
Наука и техника    53136
Бизнес              7399
Name: topic, dtype: int64

In [0]:

def train_test_split(source_col, target_col, source_features_encoder):
    X = source_features_encoder(source_col)
    y = target_col
    
    training_size = len(source_col)*70//100
    X_train, y_train = X[:training_size], y[:training_size]
    X_test, y_test = X[training_size:], y[training_size:]
    
    return X_train, y_train, X_test, y_test


def run_experiment(dataset, dataset_part_size, source_features_encoder):
    
    
    print(len(dataset))
    precision_scores = []
    recall_scores = []
    f1_scores = []

    for i in range(5):
        dataset_shuf = dataset.sample(frac=1)[:dataset_part_size]
        print(len(dataset_shuf))

        X_train, y_train, X_test, y_test = train_test_split(dataset_shuf.text, dataset_shuf.topic_cat, source_features_encoder)

        clf = LogisticRegression()
        clf.fit(X_train, y_train)

        predictions = clf.predict(X_test)

        precision = precision_score(y_test.values, predictions, average='weighted')
        recall = recall_score(y_test.values, predictions, average='weighted')
        f1 = f1_score(y_test.values, predictions, average='weighted')

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

        print(f"precision {str(precision)}")
        print(f"recall {str(recall)}")
        print(f"f1-score {str(f1)}")

    print('*' * 10)
    print("mean precision", np.mean(precision_scores))
    print("mean recall", np.mean(recall_scores))
    print("mean f1-score", np.mean(f1_scores))

### наивное решение

In [0]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))

def naive_features_tfidf_encoder(text_col):
    return tfidf_vectorizer.fit_transform(text_col)

In [9]:
run_experiment(lenta, 5000, naive_features_tfidf_encoder)


258297
5000
precision 0.9084456093203755
recall 0.9333333333333333
f1-score 0.9196198747258548
5000
precision 0.8988434882191244
recall 0.924
f1-score 0.9104899256324784
5000
precision 0.9175120999762175
recall 0.9373333333333334
f1-score 0.9261941123947085
5000
precision 0.9133043288950023
recall 0.9366666666666666
f1-score 0.9238255365481983
5000
precision 0.9041574823929077
recall 0.9266666666666666
f1-score 0.9141783018186467
**********
mean precision 0.9084526017607255
mean recall 0.9316000000000001
mean f1-score 0.9188615502239774


**Ого!**

### данные посложнее

In [0]:
!wget -q https://github.com/BobaZooba/HSE-Deep-Learning-in-NLP-Course/blob/master/week_05/data/answers_subsample.csv?raw=true -O data.csv

In [11]:
data = pd.read_csv('data.csv')
data.columns=['topic', 'text']
data["topic_cat"] = data.topic.astype('category').cat.codes
data.head()

,topic,text,topic_cat
0,business,Могут ли в россельхозбанке дать в залог норков...,0
1,law,Может ли срочник перевестись на контракт после...,2
2,business,Продажа недвижимости по ипотеки ? ( арестованы...,0
3,business,"В чем смысл криптовалюты, какая от неё выгода ...",0
4,law,часть 1 статья 158 похитил телефон,2


In [12]:
run_experiment(data, 50000, naive_features_tfidf_encoder)


237779
50000
precision 0.7646808063770543
recall 0.7598
f1-score 0.7578620011727235
50000
precision 0.758751928639502
recall 0.7538666666666667
f1-score 0.7518202673490374
50000
precision 0.7636267117739872
recall 0.7576
f1-score 0.7551476243253603
50000
precision 0.7592233581815434
recall 0.7536666666666667
f1-score 0.751759271917705
50000
precision 0.7665396748384431
recall 0.7607333333333334
f1-score 0.75861358429415
**********
mean precision 0.7625644959621061
mean recall 0.7571333333333333
mean f1-score 0.7550405498117952


### может, нам поможет морфология?


In [13]:
morph_analyzer = pymorphy2.MorphAnalyzer()
russian_stopwords = stopwords.words('russian')

tokenization_results = {}

def pymorphy_preprocess_tokenize(text):
    
    text_preprocessed_tokenized = []
        
    for sentence in ru_sent_tokenize(text):
        
        clean_words = [word.strip(string.punctuation) for word in word_tokenize(text)]
        clean_words = [word for word in clean_words if word]
        clean_words = [word.lower() for word in clean_words if word]
        clean_words = [word for word in clean_words if word not in russian_stopwords]
        
        clean_lemmas = []
        for word in clean_words:
            lemma = tokenization_results.get(word, None) 
            if lemma is None:
                lemma = morph_analyzer.parse(word)[0].normal_form
            tokenization_results[word] = lemma
            clean_lemmas.append(lemma)
        
        text_preprocessed_tokenized.extend(clean_lemmas)

    return text_preprocessed_tokenized

pymorphy_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), tokenizer=pymorphy_preprocess_tokenize)
def pymorphy_features_tfidf_encoder(text_col):
    return pymorphy_tfidf_vectorizer.fit_transform(text_col)

run_experiment(data, 50000, pymorphy_features_tfidf_encoder)

237779
50000
precision 0.8012186882590834
recall 0.7970666666666667
f1-score 0.7968329866383274
50000
precision 0.8027384268036231
recall 0.7992
f1-score 0.7985790636422943
50000
precision 0.7977775029844452
recall 0.7941333333333334
f1-score 0.7930188030910998
50000
precision 0.797166170909154
recall 0.794
f1-score 0.7930872338867725
50000
precision 0.797412569862217
recall 0.7938666666666667
f1-score 0.7932143615796704
**********
mean precision 0.7992626717637046
mean recall 0.7956533333333333
mean f1-score 0.7949464897676328


In [14]:
run_experiment(data, 1000, naive_features_tfidf_encoder)

237779
1000
precision 0.5311291927618388
recall 0.42333333333333334
f1-score 0.39047818615732827
1000
precision 0.5810634293242989
recall 0.38666666666666666
f1-score 0.2977149296973708
1000
precision 0.5367994294567656
recall 0.44666666666666666
f1-score 0.3862334715244727
1000
precision 0.46347095554044654
recall 0.46
f1-score 0.39365359937272576
1000
precision 0.5123649389825861
recall 0.36666666666666664
f1-score 0.2954115122207227
**********
mean precision 0.5249655892131871
mean recall 0.4166666666666667
mean f1-score 0.352698339794524


In [15]:
run_experiment(data, 1000, pymorphy_features_tfidf_encoder)

237779
1000
precision 0.647496431598198
recall 0.49333333333333335
f1-score 0.4519528861679891
1000
precision 0.6497538086115672
recall 0.49666666666666665
f1-score 0.4623174508963028
1000
precision 0.6383625509972038
recall 0.47
f1-score 0.4309059671908212
1000
precision 0.6535286749928223
recall 0.53
f1-score 0.5046924786249203
1000
precision 0.6608987943003288
recall 0.49333333333333335
f1-score 0.4672680676851558
**********
mean precision 0.6500080521000241
mean recall 0.4966666666666667
mean f1-score 0.4634273701130378


### rnnmorph

In [16]:

predictor = RNNMorphPredictor(language="ru")



def rnnmorph_preprocess_tokenize(text):
    
    text_preprocessed_tokenized = []
        
    for sentence in ru_sent_tokenize(text):
        
        clean_words = [word.strip(string.punctuation) for word in word_tokenize(text)]
        clean_words = [word for word in clean_words if word]
        clean_words = [word.lower() for word in clean_words if word]
        
        clean_lemmas = [analysis.pos + '_' + analysis.normal_form for analysis in predictor.predict(clean_words)]
        text_preprocessed_tokenized.extend(clean_lemmas)

    return text_preprocessed_tokenized



rnnmorph_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), tokenizer=rnnmorph_preprocess_tokenize)
def rnnmorph_features_tfidf_encoder(text_col):
    return rnnmorph_tfidf_vectorizer.fit_transform(text_col)

run_experiment(data, 1000, rnnmorph_features_tfidf_encoder)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.






237779
1000
precision 0.6489297444847184
recall 0.5266666666666666
f1-score 0.4774705668861379
1000
precision 0.6196762347568798
recall 0.49
f1-score 0.4558070024481685
1000
precision 0.6083589225589225
recall 0.5133333333333333
f1-score 0.4657759975144933
1000
precision 0.5786585480407944
recall 0.49
f1-score 0.4618377244407359
1000
precision 0.6094785353535354
recall 0.56
f1-score 0.5400399389844786
**********
mean precision 0.6130203970389702
mean recall 0.5159999999999999
mean f1-score 0.4801862460548028
